In [7]:
import pymysql  
  
# 创建连接  
connection = pymysql.connect(  
    host='localhost',  # 数据库服务器地址  
    user='root',   # 数据库用户名  
    password='root', # 数据库密码  
    database='bilibili',  # 数据库名  
    charset='utf8mb4',  # 字符集  
    cursorclass=pymysql.cursors.DictCursor  # 返回字典形式的游标  
)  
  
try:  
    with connection.cursor() as cursor:  
        # 在此处执行 SQL 查询  
        sql = "SELECT `id`, `username` FROM `test` "  
        cursor.execute(sql)  
        result = cursor.fetchone()  
        print(result)  
finally:  
    connection.close()

{'id': 1, 'username': 'bob'}


In [32]:
import requests
import pymysql

# 创建连接  
connection = pymysql.connect(  
    host='localhost',  # 数据库服务器地址  
    user='root',   # 数据库用户名  
    password='root', # 数据库密码  
    database='bilibili',  # 数据库名  
    charset='utf8mb4',  # 字符集  
    cursorclass=pymysql.cursors.DictCursor  # 返回字典形式的游标  
)  

# 数据接口的URL  
url = 'https://api.bilibili.com/x/space/wbi/arc/search?mid=51420401&pn=1&ps=25&index=1&order=pubdate&order_avoided=true&platform=web&web_location=1550101&dm_img_list=[%7B%22x%22:550,%22y%22:-87,%22z%22:0,%22timestamp%22:4,%22k%22:107,%22type%22:0%7D,%7B%22x%22:951,%22y%22:-634,%22z%22:48,%22timestamp%22:161,%22k%22:88,%22type%22:0%7D]&dm_img_str=V2ViR0wgMS4wIChPcGVuR0wgRVMgMi4wIENocm9taXVtKQ&dm_cover_img_str=QU5HTEUgKEFwcGxlLCBBcHBsZSBNMSwgT3BlbkdMIDQuMSlHb29nbGUgSW5jLiAoQXBwbG&dm_img_inter=%7B%22ds%22:[%7B%22t%22:0,%22c%22:%22%22,%22p%22:[306,102,102],%22s%22:[81,4951,4822]%7D],%22wh%22:[3508,5511,26],%22of%22:[197,394,197]%7D&w_rid=65cd76a1cb55f0c43636f374285832b0&wts=1713265842'  
  
# 设置请求头，模拟浏览器行为  
headers = {  
    # 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'
}  
  
# 如果需要处理Cookies，可以从浏览器获取后设置  
# cookies = {'cookie_name': 'cookie_value'}  
  
# 发送GET请求，带上请求头  
response = requests.get(url, headers=headers)  # 如果需要带上Cookies，可以加上cookies=cookies参数  

# 检查请求是否成功  
if response.status_code == 200:  
    # 解析JSON响应（如果接口返回的是JSON）  
    json_data = response.json()
    # 获取节目列表
    data=json_data["data"]["list"]['vlist']
    print(type(data))
    
    try:
        with connection.cursor() as cursor: 
            # 在此处执行 SQL 查询 
            for item in data:
                sql = "INSERT INTO updata (video_name, play_count, updated, picture, description, comment, bvid) VALUES (%s, %s, %s, %s, %s, %s, %s)" 
                print(item["title"])
                print(type(item["title"]))
                values=(item["title"], 
                        item["play"], 
                        item["created"], 
                        item["pic"], 
                        item["description"], 
                        item["comment"], 
                        item["bvid"])
                cursor.execute(sql, values) 
            connection.commit()  
    finally:
        connection.close()
        print("Data inserted successfully.") 
else:  
    print(f"请求失败，状态码：{response.status_code}")

<class 'list'>
【开源与商业】谁用谁维护，Redis 要赚钱！
<class 'str'>
写代码还给钱？开源之夏了解一下？
<class 'str'>
【树莓派】10年，累计销售量5000万，估值5亿美元，树莓派存在的意义是什么？
<class 'str'>
15000元的RISC-V台式工作站能做点什么？RISC-V的软件生态发展的如何？
<class 'str'>
芯片战国时代，操作系统应该大一统？？？
<class 'str'>
中美科技战后，中国如何建设自己的 Linux操作系统？
<class 'str'>
Data inserted successfully.


In [101]:
import requests  
from bs4 import BeautifulSoup  
  
# 发送请求，获取网页内容  
url = 'https://remword.com/kps_result/'  # 这里替换为你想要抓取的网页的 URL  
response = requests.get(url)  
  
# 使用 BeautifulSoup 解析网页内容  
soup = BeautifulSoup(response.text, 'html.parser')  
  
# 查找你想要的数据，这里以查找所有的段落为例  
tr_tags = soup.find_all('table')[2].find_all('tr')

for tr in tr_tags:
    tds = tr.find_all('td')
    if len(tds)>1:
        plist=tds[0].find_all('p')
    if len(tds)>=2:
        alist=tds[1].find('a')
        if alist is not None:
            print("https://remword.com/kps_result"+alist.get('href').lstrip('.')  )
# 获取 url 链接
#second_p_in_first_td = [tr.find('td').find('p') for tr in tr_tags if tr.find('td')]


# 获取 Linux Kernel 版本
# first_p_in_first_td = [tr.find('td').find('p') for tr in tr_tags if tr.find('td')]
# #打印出所有的段落文本 
# for p in first_p_in_first_td:  
#     if p:
#         print(p.text)

https://remword.com/kps_result/all_whole.html
https://remword.com/kps_result/6.9_whole.html
https://remword.com/kps_result/6.8_whole.html
https://remword.com/kps_result/6.7_whole.html
https://remword.com/kps_result/6.6_whole.html
https://remword.com/kps_result/6.5_whole.html
https://remword.com/kps_result/6.4_whole.html
https://remword.com/kps_result/6.3_whole.html
https://remword.com/kps_result/6.2_whole.html
https://remword.com/kps_result/6.1_whole.html
https://remword.com/kps_result/6.0_whole.html
https://remword.com/kps_result/5.19_whole.html
https://remword.com/kps_result/5.18_whole.html
https://remword.com/kps_result/5.17_whole.html
https://remword.com/kps_result/5.16_whole.html
https://remword.com/kps_result/5.15_whole.html
https://remword.com/kps_result/5.14_whole.html
https://remword.com/kps_result/5.13_whole.html
https://remword.com/kps_result/5.12_whole.html
https://remword.com/kps_result/5.11_whole.html
https://remword.com/kps_result/5.10_whole.html
https://remword.com/kps_